In [1]:
!pip install --quiet transformers torch scikit-learn
!pip install --quiet pandas numpy tqdm

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import pandas as pd
import json
import pickle
import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# Cấu hình và thiết lập

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
WINDOW_SIZE = 60  # Theo bài báo: long-term trend
EMBED_DIM = 48    # PCA: 768 → 48
BATCH_SIZE = 8
INPUT_JSON = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/X.json"
OUTPUT_NPZ = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/preprocessed_data.npz"
SCALERS_PKL = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/scalers.pkl"

print(f"[INFO] Thiết bị: {DEVICE}")
print(f"[INFO] Window size: {WINDOW_SIZE} ngày (theo bài báo gốc)")

[INFO] Thiết bị: cpu
[INFO] Window size: 60 ngày (theo bài báo gốc)


## phoBERT

In [4]:
print("[INFO] Đang tải PhoBERT...")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
bert_model = AutoModel.from_pretrained("vinai/phobert-base").to(DEVICE)
bert_model.eval()

[INFO] Đang tải PhoBERT...


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [5]:
pca = PCA(n_components=EMBED_DIM)
news_scaler = StandardScaler()

In [6]:
EVENT_KEYWORDS = ["hội nghị", "lễ hội", "APEC", "triển lãm", "đại hội", "du lịch", "hội thảo", "triển lãm", "festival"]


# Tính toán đặc trưng

In [7]:
def detect_event_score(text):
    """Tăng độ nhạy với sự kiện (theo bài báo)"""
    if pd.isna(text) or not text.strip():
        return 0
    text = text.lower()
    score = sum(1 for kw in EVENT_KEYWORDS if kw in text)
    return min(score, 3)  # cap tại 3

def get_bert_embedding(text):
    """BERT embedding với xử lý missing"""
    if pd.isna(text) or not text.strip():
        return np.zeros(768)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True).to(DEVICE)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy().squeeze()

def load_data(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    rows = []
    for date, item in data.items():
        if "error" in item or "hotel" not in item:
            print(f"[WARNING] Thiếu 'hotel' tại {date} → Bỏ qua ngày này.")
            continue
        hotel = item["hotel"]
        env = item["env"]
        rows.append({
            "date": date,
            "single_price": hotel["single"]["price"],
            "single_available": hotel["single"]["available"],
            "single_sold": hotel["single"]["sold"],
            "double_price": hotel["double"]["price"],
            "double_available": hotel["double"]["available"],
            "double_sold": hotel["double"]["sold"],
            "vip_price": hotel["vip"]["price"],
            "vip_available": hotel["vip"]["available"],
            "vip_sold": hotel["vip"]["sold"],
            "revenue": hotel["revenue"],
            "is_weekend": env["is_weekend"],
            "is_holiday": env["is_holiday"],
            "holiday_score": env["holiday_score"],
            "local_event": env["local_event"],
            "event_score": env["event_score"],
            "weather_score": env["weather_score"],
            "comp_price_single": env["competitor_avg_single_price"],
            "comp_price_double": env["competitor_avg_double_price"],
            "comp_price_vip": env["competitor_avg_vip_price"],
            "news": env["news"]
        })
    df = pd.DataFrame(rows)
    df["date"] = pd.to_datetime(df["date"], format="%d-%m-%y")
    df = df.sort_values("date").reset_index(drop=True)
    print(f"[INFO] Đã tải {len(df)} ngày hợp lệ từ {df['date'].min()} đến {df['date'].max()}")
    return df

def add_derived_features(df):
    # Tính occupancy, free
    for room in ["single", "double", "vip"]:
        df[f"{room}_occupancy"] = df[f"{room}_sold"] / (df[f"{room}_sold"] + df[f"{room}_available"] + 1e-8)
        df[f"{room}_free"] = df[f"{room}_available"]

    # Event score từ news
    df["event_score_detected"] = df["news"].apply(detect_event_score)

    # Tổng event score
    df["event_score_final"] = df["event_score"] + df["event_score_detected"]

    return df, None

# Embeding



In [8]:
def embed_news_and_pca(df):
    print("[INFO] Bắt đầu embedding news với BERT...")
    embeddings = []
    for i, text in enumerate(df["news"]):
        if i % 50 == 0:
            print(f"   Đang xử lý: {i}/{len(df)}")
        emb = get_bert_embedding(text)
        embeddings.append(emb)
    embeddings = np.array(embeddings)  # (N, 768)

    print(f"[INFO] Kích thước news embedding trước PCA: {embeddings.shape}")

    # PCA
    news_pca = pca.fit_transform(embeddings)

    # SCALE
    news_pca_scaled = news_scaler.fit_transform(news_pca)

    print(f"[INFO] Hoàn tất PCA + Scale → {news_pca_scaled.shape}")

    # Gán lại vào df
    for i in range(EMBED_DIM):
        df[f"news_emb_{i}"] = news_pca_scaled[:, i]

    return df

# TẠO CỬA SỔ TRƯỢT (SLIDING WINDOW)

In [9]:
def create_sliding_windows(df):
    feature_cols = [
        'single_price', 'single_available', 'single_sold', 'single_occupancy', 'single_free',
        'double_price', 'double_available', 'double_sold', 'double_occupancy', 'double_free',
        'vip_price', 'vip_available', 'vip_sold', 'vip_occupancy', 'vip_free',
        'comp_price_single', 'comp_price_double', 'comp_price_vip',
        'is_weekend', 'is_holiday', 'holiday_score', 'event_score_final', 'weather_score'
    ]  # 23 đặc trưng/phòng × 3 = 69? Không, mỗi phòng 8 → 24 + 8 chung = 32?

    # Thực tế: 11 đặc trưng/phòng (price, avail, sold, occ, free, comp_price) × 3 + 8 env = 41
    # + 48 news_emb → d = 89
    # → Nhưng RPT dùng d=59 → ta sẽ lấy 11/phòng + 8 env + 40 news (sau PCA chọn 40)

    # CHUẨN HÓA: 11 đặc trưng/phòng
    room_features = ['price', 'available', 'sold', 'occupancy', 'free', 'comp_price_single', 'comp_price_double', 'comp_price_vip'][:5] + ['comp_price']
    env_features = ['is_weekend', 'is_holiday', 'holiday_score', 'event_score_final', 'weather_score']

# Tạo X
    X_list = []
    y_list = []        # ← BÂY GIỜ LÀ (3,) – số phòng bán ra
    dates_list = []

    # CHỈ CẦN price_scaler (cho input)
    price_scaler = MinMaxScaler()

    # Fit scaler trên toàn bộ giá
    all_prices = []
    for room in ["single", "double", "vip"]:
        all_prices.extend(df[f"{room}_price"].values)
    price_scaler.fit(np.array(all_prices).reshape(-1, 1))

    N = len(df)
    for i in range(WINDOW_SIZE, N):
        window = df.iloc[i - WINDOW_SIZE:i]
        target_date = df.iloc[i]["date"]

        # === TẠO X: (T, 3, d) ===
        X_window = np.zeros((WINDOW_SIZE, 3, 11 + len(env_features) + EMBED_DIM))

        for t in range(WINDOW_SIZE):
            row = window.iloc[t]
            for r, room in enumerate(["single", "double", "vip"]):
                X_window[t, r, 0] = price_scaler.transform([[row[f"{room}_price"]]])[0, 0]
                X_window[t, r, 1] = row[f"{room}_available"]
                X_window[t, r, 2] = row[f"{room}_sold"]
                X_window[t, r, 3] = row[f"{room}_occupancy"]
                X_window[t, r, 4] = row[f"{room}_free"]
                X_window[t, r, 5] = row[f"comp_price_{room}"] if f"comp_price_{room}" in df.columns else 0
            # Env features (chung cho 3 phòng)
            for e_idx, e_feat in enumerate(env_features):
                X_window[t, :, 6 + e_idx] = row[e_feat]
            # News embedding (chung)
            for emb_idx in range(EMBED_DIM):
                X_window[t, :, 11 + len(env_features) + emb_idx] = row[f"news_emb_{emb_idx}"]

        # === TẠO y: số phòng bán ra ngày mai (3 giá trị) ===
        target_row = df.iloc[i]
        y_sold = [
            target_row["single_sold"],
            target_row["double_sold"],
            target_row["vip_sold"]
        ]

        X_list.append(X_window)
        y_list.append(y_sold)
        dates_list.append(target_date)

    X = np.array(X_list, dtype=np.float32)
    y = np.array(y_list, dtype=np.float32)  # Shape: (N, 3)

    print(f"[INFO] Tạo {len(X)} cửa sổ trượt:")
    print(f"     X ∈ ℝ^({X.shape[0]}, {WINDOW_SIZE}, 3, {X.shape[-1]})")
    print(f"     y ∈ ℝ^({y.shape[0]}, 3) ← [sold_single, sold_double, sold_vip]")

    return X, y, dates_list, {
        'price': price_scaler,
        'news': news_scaler
    }

# Main
Thể hiện quy trình embbed cũng như preprocessing của nhóm

In [10]:
def main():
    print("=== BẮT ĐẦU GIAI ĐOẠN: NEWS EMBEDDING + PREPROCESS (CẢI THIỆN) ===")

    # 1. Load
    df = load_data(INPUT_JSON)

    # 2. Add features
    df, _ = add_derived_features(df)

    # 3. Embed + PCA + Scale
    df = embed_news_and_pca(df)

# 4. Sliding window – ĐÃ SỬA: y = [sold_single, sold_double, sold_vip]
    X, y_demand, dates, scalers = create_sliding_windows(df)

    # 5. Save – ĐÃ SỬA tên biến
    np.savez_compressed(
        OUTPUT_NPZ,
        X=X,
        y_demand=y_demand,                    # ← ĐỔI TÊN
        dates=np.array(dates, dtype='datetime64[D]'),
        feature_cols_per_room=np.array([
            'price_scaled', 'available', 'sold', 'occupancy', 'free',
            'comp_price', 'is_weekend', 'is_holiday', 'holiday_score',
            'event_score_final', 'weather_score'
        ] + [f"news_emb_{i}" for i in range(EMBED_DIM)])
    )

    with open(SCALERS_PKL, 'wb') as f:
        pickle.dump(scalers, f)

    print(f"[SUCCESS] Đã lưu:")
    print(f"   → {OUTPUT_NPZ}")
    print(f"   → {SCALERS_PKL}")
    print(f"   - Số mẫu: {X.shape[0]}")
    print(f"   - Kích thước X: {X.shape}")
    print(f"   - d = {X.shape[-1]} đặc trưng/phòng")
    print(f"   - y_demand shape: {y_demand.shape} ← [sold_single, sold_double, sold_vip]")
    print("=== HOÀN TẤT – THEO ĐÚNG BÀI BÁO GỐC (Mei, 2025) ===")

    return X, y_demand, dates

# 6. HÀM CHÍNH

In [11]:
if __name__ == "__main__":
    X, y_demand, dates = main()  # ← y_demand = [sold_single, sold_double, sold_vip]

    # DEBUG – THEO ĐÚNG BÀI BÁO: DỰ ĐOÁN DEMAND
    print("\n[DEBUG] Mẫu đầu tiên – DỰ ĐOÁN DEMAND (SỐ PHÒNG BÁN RA)")
    print(f"   Ngày dự đoán: {dates[0]}")
    print(f"   Số phòng thực tế đã bán:")
    print(f"     • Single: {int(y_demand[0, 0])} phòng")
    print(f"     • Double: {int(y_demand[0, 1])} phòng")
    print(f"     • VIP:    {int(y_demand[0, 2])} phòng")
    print(f"   Shape X[0]: {X[0].shape} ← (60, 3, d)")
    print(f"   Giá single ngày cuối (scaled): {X[0, -1, 0, 0]:.4f}")
    print(f"   News embedding ngày cuối (phòng 0): {X[0, -1, 0, -EMBED_DIM:][:3]}... (48 dims)")

=== BẮT ĐẦU GIAI ĐOẠN: NEWS EMBEDDING + PREPROCESS (CẢI THIỆN) ===
[WARNING] Thiếu 'hotel' tại 09-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 12-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 14-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 24-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 27-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 31-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 13-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 22-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 25-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 27-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 28-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 01-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 02-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 07-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 08-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 09-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 10-03-24 